<a href="https://colab.research.google.com/github/akiran703/DQN_demo_for_SI/blob/main/DQN_for_spaceinvaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.8.1
    Uninstalling ale-py-0.8.1:
      Successfully uninstalled ale-py-0.8.1


In [31]:
import gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [33]:
from ale_py import ALEInterface
ale = ALEInterface()

from ale_py.roms import SpaceInvaders
ale.loadROM(SpaceInvaders)

In [75]:
#function to create an environment

def create_env():
  base_env = gym.make("ALE/SpaceInvaders-v5")
  env = gym.wrappers.GrayScaleObservation(base_env)
  env = gym.wrappers.FrameStack(env,4)
  env = gym.wrappers.ResizeObservation(env, (84,84))
  #print(base_env.action_space)
  #print(env.observation_space)

  return env




real_env = create_env()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [77]:
# if GPU is to be used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [78]:
#in order to reduce instability for when the Deep Q-learning is being trained, we apply a replay system to help.
#A replay system essentially is a way to store transistions that the agent observes and it can be used later training.
#This helps with reducing catostraphic forgetting in which the agent forgets all previous experiences as its new experience and reduces correlation between experiences.


#create a tuple to define the states
Transition = namedtuple('Transition',('state', 'action', 'next_state', 'reward'))

#creating that class that creates the Replay system
class ReplayMemory(object):
    #constructor creating a queue to store the experiences
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
    # adding the experience to the queue
    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))
    # getting a random experience
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    # the length of the queue
    def __len__(self):
        return len(self.memory)
